In [1]:
#Biblioteket
import numpy as np

In [2]:
#Enklar deklaratioer
m= 1000     #(kg) är massan
mu=3        #Friktionskoffecient 2=<mu<=4, vi valde medelvärdet
g= 9.81     #(m/s^2) tyngdfaktor

Cd= 0.3     #luftmotståndskoeffecient ENHETSLÖS!
Af= 1.63    #Area-front ENHETSLÖS!
p_air=1.2   #(kg/m^3)   luftdensitet 
v_veh=150   #(m/s) Hastigheten som vi skall VARIERAR ENLIGT ÖNSKEMÅL

C_rr= 0.03   #rullmotståndskoffecient 
alpha= 0    #(grader) lutningen i grander VARIERAR ENLIGT ÖNSKEMÅL

a_veh=10    #(m/s^2) bilens acceleration vid nån-tidspunkt VARIERAR ENLIGT ÖNSKEMÅL

In [ ]:
#Implementering av ekvationerna som vi har från In 2 del (TEKNISKA KRAV)

a_max= mu * g       #(m/s^2) Max acceleration innan slieringen
print("a_max=",a_max)

F_veh= m * a_max    #(N) Maximala kraften vi kan ha innan däck slirar
print("F_veh=", F_veh)        

def F_aero(v_veh):  #(N) Luftmotstånds-kraft
    F_aero = (Cd * Af * p_air * (v_veh**2)) / 2 
    return F_aero
print("F_aero(v_veh)=",F_aero(v_veh))

def F_roll(alpha, v_veh, m):   #(N) Friktionskraft
    F_roll = 0
    if v_veh == 0:
        F_roll = 0
    else:
        F_roll = C_rr * m * g * np.cos(alpha) 
    return F_roll
print("F_roll(alpha, v_veh)=",F_roll(alpha, v_veh, m))

def F_grad(alpha, m):     #(N) Den vertikala gradientens kraftkomposant 
    F_grad= m * g * np.sin(alpha)
    return F_grad
print("F_grad(alpha)=",F_grad(alpha,m))

F_net= m * a_veh        #(N) Den resulterande kraften 
print("F_net=",F_net)

#F_net=F_trac - F_roll - F_grad --> F_trac= F_aero + F_roll + F_grad + F_net 
F_trac = F_aero(v_veh) + F_roll(alpha, v_veh) + F_grad(alpha) + F_net    #(N) Den drivandekraften från motor till hjulen
print("F_trac=",F_trac)

a_max= 29.43
F_veh= 29430.0
F_aero(v_veh)= 6601.499999999998
F_roll(alpha, v_veh)= 294.3
F_grad(alpha)= 0.0
F_net= 10000
F_trac= 16895.8


In [ ]:
#Implementering av ekvationerna som vi har från In 2 del (FYSIKALISK MODELL)

#----------------------Enkla Deklarationer--------------------------------------
lamda= 1                    #Konstant ENHETSLÖS!
Ra=0.039 * 2.325            #(Ohm) Resistansten 
r_hjul= 0.46482             #(m) 36.6 (inches) / 2 *0.0254 = 0.46482 (m)
T_f= 0                      #(Nm) Friktionsförluster i motor VI BORTSER FRÅN DE 
T_L= 0                      #(Nm) Lastmoment    VARIERAR ENLIGT ÖNSKEMÅL
n= 0.982                    #Verkningsgrad ENHETSLÖS!
i= 5                        #Utväxlingen ENHETSLÖS! (Vi valde planet Växel)

#-----------------------Ekvationer----------------------------------------------
def T_e(F_trac):            #(Nm) Motorvridmoment
    T_e= ((F_trac * r_hjul)/n * i) + T_f + T_L
    return T_e
print("T_e(F_trac)=", T_e(F_trac))

def w_r(v_veh):             #(rad/s) varvtal för elmotorn
    w_r= i * (v_veh/r_hjul)     
    return w_r
print("w_r(v_veh)=",w_r(v_veh))

def ia(F_trac, v_veh):      #(A) Stationär ankarsström 
    ia= (1/lamda) * T_e(F_trac)
    return ia
print("ia(F_trac, v_veh)=",ia(F_trac, v_veh))

def U_t(F_trac, v_veh):     #(V) Stationär spänning 
    U_t= Ra * (1/lamda) * T_e(F_trac) + lamda * w_r(v_veh)
    return U_t
print("U_t(F_trac, v_veh)=",U_t(F_trac, v_veh))



T_e(F_trac)= 24466.295813767825
w_r(v_veh)= 1613.527817219569
ia(F_trac, v_veh)= 24466.295813767825
U_t(F_trac, v_veh)= 3832.0091901329665


In [ ]:
#=====================================================================
#-----------------------Simulering---------------------------------------
#=====================================================================
#-----------------Simulering funktion------------------------------------------
x = 1                           #Variabel för att bestämma spänning- eller ström styrd
U = [2679, x]                   #(v) 
I = [669, 1-x]                  #(A)

def sim(m,alpha, U, I):         #simulerig med insignalerna m, alpha, U, I 
    if U[1]==True:              #konstant spänning / dvs spänningstyrd
        simtyp = 1
        U=U[0]                  #U=2679 (V)
    else: 
        simtyp = 0              #Konstant ström / dvs strömstyrd
        I=I[0]                  #I= 669 (A)

    N = 1000                    #Antal iterrationer 
    dt= 0.001                   #(s) Tidsteg (start fråm 1ms)
    t_tot = N *dt               #(s) Totala simuleringstid 
#------------------Skapar vektorer för att lagra in resultat-------------
    t_vec = np.zeros(N)         #tid            ex.[0,0,....,0]
    v_vec = np.zeros(N)         #hastighet      ex.[0,0,....,0]
    a_vec = np.zeros(N)         #acceleration   ex.[0,0,....,0]
    ia_vec = np.zeros(N)        #ankarstörm     ex.[0,0,....,0]
    U_t_vec = np.zeros(N)       #spänning       ex.[0,0,....,0]
    s_vec = np.zeros(N)         #sträcka        ex.[0,0,....,0]

    for k in range(0,N):        #Från 0 till 999 (1000 iter.)
        #Mekaniska krafter från omvärden som verkar på fordonet
        F_aero_1 = F_aero(v_vec[k])
        F_roll_1 = F_roll(alpha,v_vec[k], m)
        F_grad_1 = F_grad(alpha,m) 
        F_motstånd= F_aero_1 + F_roll_1 + F_grad_1
        #Mekaniksa kraften vi får från motorn
        T_hjul = n*i*T_e
        F_trac = T_hjul/r_hjul
        F_net = F_trac - F_motstånd
        a = F_net/m


        
        




